**This notebook is for binary segmentation for skin lesion detection. ISIC dataset is used. I used the TensorFlow data processing pipeline (tf.io) for data preprocessing, and I augmented the training data using the albumentations library. The loss function employed a combination of BCE and dice-loss, while callbacks such as ReduceLRonPlateau were used for enhancing the training process. Additionally, earlystopping and modelcheckpoint callbacks were implemented to prevent overfitting and save the best performing model.**

# Imports and Preparing Environment

In [ ]:
import numpy as np
import tensorflow as tf 
from tensorflow.keras import layers
import os
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, Callback
from tensorflow.keras import backend as K
import albumentations as A
from functools import partial
from tqdm import tqdm

In [ ]:
seed = 42
np.random.seed(seed)

In [ ]:
from tensorflow.keras.callbacks import Callback

In [ ]:
print(tf.__version__)

In [ ]:
tf.config.list_physical_devices('GPU')

# Getting the data

In [ ]:
os.listdir('/kaggle/input/isic2018-challenge-task1-data-segmentation')

In [ ]:
train_img_folder_path = "/kaggle/input/isic2018-challenge-task1-data-segmentation/ISIC2018_Task1-2_Training_Input"
train_label_folder_path = "/kaggle/input/isic2018-challenge-task1-data-segmentation/ISIC2018_Task1_Training_GroundTruth"
val_folder_path = "/kaggle/input/isic2018-challenge-task1-data-segmentation/ISIC2018_Task1-2_Validation_Input"
test_folder_path = "/kaggle/input/isic2018-challenge-task1-data-segmentation/ISIC2018_Task1-2_Test_Input"

In [ ]:
# Checking if paths are correct
exist_condition = os.path.exists(train_img_folder_path) and os.path.exists(train_label_folder_path) and os.path.exists(val_folder_path) and os.path.exists(test_folder_path)
print("Files not found") if exist_condition == False else None

# Visualize the data

In [ ]:
# This array contains paths for the images. Folder contains a text file, we only want images I used endswith function to get jpg files
train_images_path = np.sort([os.path.join(train_img_folder_path, i) for i in os.listdir(train_img_folder_path) if i.endswith('.jpg')])
train_labels_path = np.sort([os.path.join(train_label_folder_path, i) for i in os.listdir(train_label_folder_path) if i.endswith('.png')])
test_images_path = np.sort([os.path.join(test_folder_path, i) for i in os.listdir(test_folder_path) if i.endswith('.jpg')])

In [ ]:
train_split = 0.8
val_images_path = train_images_path[int(train_split*len(train_images_path)):]
val_labels_path = train_labels_path[int(train_split*len(train_labels_path)):]
train_images_path = train_images_path[:int(train_split*len(train_images_path))]
train_labels_path = train_labels_path[:int(train_split*len(train_labels_path))]

In [ ]:
len(train_images_path) + len(val_images_path)

In [ ]:
# Visualizing first train image with its label.
img1 = Image.open(train_images_path[0])
img1.thumbnail((512, 512))
img1_label = Image.open(train_labels_path[0])
img1_label.thumbnail((512, 512))
plt.subplots(figsize=(6,3))
plt.subplot(1,2,1)
plt.imshow(img1)
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(img1_label, cmap='gray')
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# This function plots 25 random images with segmantation mask.
def plot_segmentation_images(image_path, image_label_path, size=512, Random=True):
    figure = plt.subplots(figsize=(25, 25))
    
    # plotting rows iterating through 
    for i in range(25): 
        random_number = np.random.randint(len(image_path)) if Random else i
        img = Image.open(image_path[random_number])
        img_label = Image.open(image_label_path[random_number])
        img = img.resize((size, size))
        img_label = img_label.resize((size, size))
        img = np.array(img)
        img_label = np.array(img_label)
        mask = np.ma.masked_where(img_label>0, img_label)
        img_label_mask = np.ma.masked_array(img_label, mask)
        img_name = image_label_path[random_number].split('/')[-1]

        plt.subplot(5, 5, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(img, cmap='gray')
        plt.imshow(img_label_mask, cmap='jet', alpha=0.4)
        plt.xlabel(f'ID: {img_name}', fontsize=15)
    plt.show()

In [ ]:
# Plots 25 random train images.
plot_segmentation_images(train_images_path, train_labels_path)

# Loading The Data

Few notes before starting. In a previous version of my code, I used Imagedatagenerator(). However, this method is now deprecated and not recommended by TensorFlow documentation. Additionally, it is much slower than using tf.Data, which is what I am using in this version of my code.

I will also be using a portion of the training data as the validation input since there is no validation ground truth in the dataset. Finally, I will be utilizing the pipelining features of tf.data even though the dataset is relatively small, as it can help improve performance and scalability in larger datasets.

In [ ]:
IMG_SIZE = 256
IMG_CHANNELS = 3
batch_size = 8
buffer_size = 1000
steps_per_epoch = len(train_images_path) // batch_size
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
x_train_ds = tf.data.Dataset.list_files(train_images_path, seed=seed)
y_train_ds = tf.data.Dataset.list_files(train_labels_path, seed=seed)
x_val_ds = tf.data.Dataset.list_files(val_images_path, seed=seed)
y_val_ds = tf.data.Dataset.list_files(val_labels_path, seed=seed)

In [ ]:
def parse_image(img_path, size=(256,256)):
    image = tf.io.read_file(img_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size)
    return image

def parse_gt(gt_path, size=(256,256)):
    gt = tf.io.read_file(gt_path)
    gt = tf.image.decode_png(gt, channels=1)
    gt = tf.image.resize(gt, size, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    gt = tf.where(gt == 0, 0, 1)
    gt = tf.image.convert_image_dtype(gt, tf.int32)
    return gt

In [ ]:
x_train_ds = x_train_ds.map(parse_image)
y_train_ds = y_train_ds.map(parse_gt)
x_val_ds = x_val_ds.map(parse_image)
y_val_ds = y_val_ds.map(parse_gt)

In [ ]:
print(f'Train x size: {len(x_train_ds)}')
print(f'Validation x size: {len(x_val_ds)}')
print(f'Train y size: {len(y_train_ds)}')
print(f'Validation y size: {len(y_val_ds)}')

In [ ]:
train_ds = tf.data.Dataset.zip((x_train_ds, y_train_ds))
val_ds = tf.data.Dataset.zip((x_val_ds, y_val_ds))

In [ ]:
def augment(image, mask, size=(256, 256)):
  transforms = A.Compose([
      A.OneOf([
          A.Transpose(),
          A.VerticalFlip(),
          A.HorizontalFlip(),
          A.RandomRotate90(),
          A.NoOp()], p=0.75),

      A.ShiftScaleRotate(p=0.1),
      A.GridDistortion(p=0.1),
      A.ElasticTransform(p=0.1),

      A.OneOf([
          A.RandomBrightnessContrast(),    
          A.RandomGamma()], p=0.2)
      
    ], additional_targets={'mask': 'mask'})

  data = {"image":image, "mask":mask}
  aug_data = transforms(**data)

  aug_img = aug_data["image"]
  aug_img = tf.cast(aug_img, tf.float32)

  aug_mask = aug_data["mask"]
  # Making sure labels are binary (background or label)
  aug_mask = np.where(aug_mask == 0, 0, 1)
  aug_mask = tf.cast(aug_mask, tf.int32)
  return aug_img, aug_mask

# I got "Tensor has no attribute .numpy() method" error. To fix this I used tf.numpy_function()
def process_data(image, mask, h=256, w=256):
  image, mask = tf.numpy_function(func=augment, inp=[image, mask], Tout=(tf.float32, tf.int32))
  # The datasets loses its shape after applying a tf.numpy_function, so this is 
  # necessary for the sequential model and when inheriting from the model class.
  image.set_shape((h, w, 3))
  mask.set_shape((h, w, 1))
  return image, mask

In [ ]:
train_ds = (
    train_ds
    .cache()
    .map(partial(process_data))
    .batch(batch_size)
    .repeat()
    .prefetch(buffer_size=AUTOTUNE)
)

# Not appliyingg augmentation on validation data.
val_ds = (
    val_ds
    .cache()
    .batch(batch_size)
    .repeat()
    .prefetch(buffer_size=AUTOTUNE)
)

In [ ]:
train_ds.element_spec

In [ ]:
for image, label in train_ds.take(1):
    print(image.shape)
    print(label.shape)
    plt.subplots(figsize=(25, 25))
    for i in range(0, batch_size, 2):
        plt.subplot(batch_size, 2, i+1)
        plt.imshow(image[i])
        plt.axis('off')
        plt.subplot(batch_size, 2, i+2)
        plt.imshow(label[i], cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.show()
    break

In [ ]:
test_ds = (
    tf.data.Dataset.list_files(test_images_path)
    .map(parse_image)
    .batch(1))

# Model

In [ ]:
# Functions to build the encoder path
def conv_block(inp, filters, padding='same', activation='relu'): 
    x = Conv2D(filters, (3, 3), padding=padding, activation=activation)(inp)
    x = Conv2D(filters, (3, 3), padding=padding)(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation(activation)(x)
    return x

def encoder_block(inp, filters, padding='same', pool_stride=2, activation='relu'):
    # Encoder block of a UNet passes the result from the convolution block
    # above to a max pooling layer
    x = conv_block(inp, filters, padding, activation)
    p = MaxPooling2D(pool_size=(2, 2), strides=pool_stride)(x)
    return x, p


# Function to build decoder path
def decoder_block(inp,filters,concat_layer,padding='same'):
    # Upsample the feature maps
    x = Conv2DTranspose(filters, (2,2), strides=(2,2), padding=padding)(inp)
    x = concatenate([x, concat_layer]) # Concatenation/Skip conncetion with conjuagte encoder
    x = conv_block(x, filters) # Passed into the convolution block above
    return x
# Building the first block
def build_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS=3):
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    d1, p1=encoder_block(inputs, 64)
    d2, p2=encoder_block(p1, 128)
    d3, p3=encoder_block(p2, 256)
    d4, p4=encoder_block(p3, 512)
    mid = conv_block(p4, 1024) # Midsection
    e2 = decoder_block(mid, 512, d4) # Conjugate of encoder 4
    e3 = decoder_block(e2, 256, d3) # Conjugate of encoder 3
    e4 = decoder_block(e3, 128, d2) # Conjugate of encoder 2 
    # o1 = Conv2D(1, (1,1), activation=None)(e4) # Output from 2nd last decoder
    e5 = decoder_block(e4, 64, d1) # Conjugate of encoder 1
    outputs = Conv2D(1, (1, 1),activation='sigmoid')(e5) #Final Output
    model = tf.keras.Model(inputs=[inputs], outputs=[outputs], name='Unet')
    return model

In [ ]:
# Metrics
def dice_coef(y_true, y_pred):
    smooth = 1e-7
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def jacard(y_true, y_pred):

    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum ( y_true_f + y_pred_f - y_true_f * y_pred_f)

    return intersection/union

In [ ]:
# Loss

def bce_dice_loss(y_true, y_pred):
    y_true = K.cast(y_true, dtype=y_pred.dtype)
    bce = K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)
    dice = dice_coef(y_true, y_pred)
    return bce - K.log(dice)

def bce_dice_loss_log(y_true, y_pred):
    y_true = K.cast(y_true, dtype=y_pred.dtype)
    bce = K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)
    dice = dice_coef(y_true, y_pred)
    return bce + 1 - dice

In [ ]:
model = build_model(IMG_SIZE, IMG_SIZE, IMG_CHANNELS)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=bce_dice_loss_log, 
              metrics=[dice_coef, jacard, 'accuracy'])
# model.summary()

In [ ]:
steps_per_epoch = int(len(train_images_path)*0.8 // batch_size)
val_steps = int(len(train_images_path)*0.2 // batch_size)
print(f"Batch Size: {batch_size}\nSteps_per_epoch: {steps_per_epoch}\nValidation_steps: {val_steps}")

In [ ]:
class DisplaySegmentationResults(Callback):
    def __init__(self, dataset, num_images=1):
        super(DisplaySegmentationResults, self).__init__()
        self.dataset = dataset
        self.num_images = num_images

    def on_epoch_end(self, epoch, logs=None):
        for i, (image, mask) in enumerate(self.dataset.take(self.num_images)):
            # Predict the mask for the image
            predicted_mask = self.model.predict(image[0][np.newaxis, ...], verbose=0)[0]
            predicted_mask = (predicted_mask > 0.5).astype(np.uint8)

            fig, ax = plt.subplots(1, 3, figsize=(6, 2))
            ax[0].imshow(image[0])
            ax[0].set_title('Input Image')
            ax[1].imshow(mask[0, ..., 0], cmap='gray')
            ax[1].set_title('Ground Truth Mask')
            ax[2].imshow(predicted_mask[..., 0], cmap='gray')
            ax[2].set_title('Predicted Mask')

            for a in ax:
                a.axis('off')

            plt.suptitle(f"Epoch {epoch+1}, Sample {i+1}")
            plt.show()

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_dice_coef', patience=10, mode='max'),
    ModelCheckpoint(filepath='saved_model.h5', monitor='val_dice_coef', save_best_only=True),
    ReduceLROnPlateau(monitor='val_dice_coef', factor=0.2, patience=5, verbose=1, min_lr=5e-7),
    DisplaySegmentationResults(val_ds)
]

In [ ]:
%%time

results = model.fit(train_ds, epochs=12, verbose=2,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=val_steps,
                    validation_data=val_ds,
                    callbacks=callbacks)

In [ ]:
plt.subplots(figsize=(8, 10))

plt.subplot(2,2,1)
plt.plot(results.history['dice_coef'])
plt.plot(results.history['val_dice_coef'])
plt.title('dice_coef vs Epoch')
plt.xlabel('Epochs')
plt.ylabel('dice_coef')
plt.legend(['train', 'validation'], loc='upper right')
plt.tight_layout()

plt.subplot(2,2,2)
plt.plot(results.history['jacard'])
plt.plot(results.history['val_jacard'])
plt.title('Jacard vs Epoch')
plt.xlabel('Epochs')
plt.ylabel('Jacard')
plt.legend(['train', 'validation'], loc='upper right')
plt.tight_layout()

plt.subplot(2,2,3)
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('Loss vs Epoch')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['train', 'validation'], loc='upper right')
plt.tight_layout()

plt.show()

In [ ]:
K.clear_session()
raw_predictions = model.predict(test_ds)

In [ ]:
predictions = np.where(raw_predictions < 0.5, 0, 1)

In [ ]:
image = [images[0].numpy() for images in test_ds.take(1)]
# Looking at an example before creating saving outputs as image.
plt.subplot(1,2,1)
plt.axis('off')
plt.imshow(image[0]) # Using index 0 because it is in shape of (1, 256, 256, 1)
plt.subplot(1,2,2)
plt.axis('off')
plt.imshow(predictions[0], cmap='gray')
plt.show()

In [ ]:
if not os.path.exists('predictions'):
    os.makedirs('predictions')

# loop through the predictions and save each one as an image
for i, pred in tqdm(enumerate(predictions)):
    pred = (pred * 255).astype(np.uint8)
    image = Image.fromarray(pred.squeeze())
    filename = f"prediction_{i}.png"
    image.save(os.path.join('predictions', filename))

In [ ]:
pred_folder_path = os.path.join(os.getcwd(), 'predictions')
pred_images_path = np.sort([os.path.join(pred_folder_path, i) for i in os.listdir(pred_folder_path) if i.endswith('.png')])
plot_segmentation_images(test_images_path, pred_images_path, size=256, Random=True)